In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False)

In [2]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
# Freeze base model
base_model.trainable = False

In [4]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(1, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs,outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# Important to use binary crossentropy and binary accuracy as we now have a binary classification problem
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

In [8]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('dataset/mask/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('dataset/mask/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

Found 2493 images belonging to 2 classes.
Found 1599 images belonging to 2 classes.


In [9]:
#model.fit(train_it,
#          validation_data=valid_it,
 #         steps_per_epoch=train_it.samples/train_it.batch_size,
  #        validation_steps=valid_it.samples/valid_it.batch_size,
   #       epochs=5)
    
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20


C:\Users\x7amo\anaconda3\envs\TensorFlow\lib\site-packages\keras\backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


12/12 [==============================] - 10s 139ms/step - loss: 0.8120 - binary_accuracy: 0.4479 - val_loss: 0.1492 - val_binary_accuracy: 0.6562
Epoch 2/20
12/12 [==============================] - 2s 127ms/step - loss: 0.5272 - binary_accuracy: 0.4271 - val_loss: 0.2636 - val_binary_accuracy: 0.5000
Epoch 3/20
12/12 [==============================] - 2s 122ms/step - loss: 0.3487 - binary_accuracy: 0.3854 - val_loss: 0.0928 - val_binary_accuracy: 0.5312
Epoch 4/20
12/12 [==============================] - 2s 123ms/step - loss: 0.4805 - binary_accuracy: 0.4271 - val_loss: 0.2466 - val_binary_accuracy: 0.6250
Epoch 5/20
 8/12 [===================>..........] - ETA: 0s - loss: 0.5023 - binary_accuracy: 0.4219

C:\Users\x7amo\anaconda3\envs\TensorFlow\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


12/12 [==============================] - 1s 120ms/step - loss: 0.4512 - binary_accuracy: 0.4583 - val_loss: 0.0871 - val_binary_accuracy: 0.5000
Epoch 6/20
12/12 [==============================] - 2s 133ms/step - loss: 0.4532 - binary_accuracy: 0.4896 - val_loss: 0.1219 - val_binary_accuracy: 0.4688
Epoch 7/20
12/12 [==============================] - 1s 120ms/step - loss: 0.3126 - binary_accuracy: 0.4271 - val_loss: 0.1067 - val_binary_accuracy: 0.4062
Epoch 8/20
12/12 [==============================] - 2s 129ms/step - loss: 0.2494 - binary_accuracy: 0.4375 - val_loss: 0.1093 - val_binary_accuracy: 0.5000
Epoch 9/20
12/12 [==============================] - 2s 130ms/step - loss: 0.2836 - binary_accuracy: 0.3854 - val_loss: 0.0249 - val_binary_accuracy: 0.5625
Epoch 10/20
12/12 [==============================] - 2s 124ms/step - loss: 0.0920 - binary_accuracy: 0.4688 - val_loss: 0.0340 - val_binary_accuracy: 0.5625
Epoch 11/20
12/12 [==============================] - 2s 123ms/step - loss:

In [10]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
#model.fit(train_it,
#          validation_data=valid_it,
#          steps_per_epoch=train_it.samples/train_it.batch_size,
#         validation_steps=valid_it.samples/valid_it.batch_size,
#        epochs=5)
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20
12/12 [==============================] - 44s 245ms/step - loss: 0.2855 - binary_accuracy: 0.5104 - val_loss: 0.0063 - val_binary_accuracy: 0.6562
Epoch 2/20
12/12 [==============================] - 3s 237ms/step - loss: 0.0560 - binary_accuracy: 0.4583 - val_loss: 0.3144 - val_binary_accuracy: 0.5000
Epoch 3/20
12/12 [==============================] - 3s 234ms/step - loss: 0.0533 - binary_accuracy: 0.3958 - val_loss: 1.8847e-04 - val_binary_accuracy: 0.5625
Epoch 4/20
12/12 [==============================] - 3s 229ms/step - loss: 0.0851 - binary_accuracy: 0.4583 - val_loss: 0.0365 - val_binary_accuracy: 0.4375
Epoch 5/20
12/12 [==============================] - 3s 229ms/step - loss: 0.2610 - binary_accuracy: 0.4375 - val_loss: 0.0469 - val_binary_accuracy: 0.6875
Epoch 6/20
12/12 [==============================] - 3s 233ms/step - loss: 0.0739 - binary_accuracy: 0.4792 - val_loss: 8.8429e-04 - val_binary_accuracy: 0.7188
Epoch 7/20
12/12 [==============================] - 5s 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224,224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

In [ ]:
make_predictions('dataset/mask/valid/mask/mask_(511).jpg')

In [ ]:
make_predictions('dataset/mask/valid/no_mask/no_mask_(1348).jpg')

In [ ]:
def mask_no_mask(image_path):
    preds = make_predictions(image_path)
    if preds[0] < 0:
        print("It's mask")
    else:
        print("That's not mask")

In [ ]:
mask_no_mask('dataset/mask/valid/no_mask/no_mask_(1328).jpg')

In [ ]:
mask_no_mask('dataset/mask/valid/mask/mask_(604).jpg')

In [1]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}